In [1]:
from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time
import copy 
import multiprocess as mp

import gym
from env import FrozenLakeCustom, FrozenLakeSimulator

from mcts_haver import run_mcts_trial
from value_iteration import value_iteration

from config import parse_args
from utils import MultiProcess

import logging
logger = logging.getLogger()
logger.setLevel(logging.FATAL)

In [2]:
np.random.seed(0)
random.seed(0)

# params
args = parse_args()

#
env_id = "FrozenLake-v1"
env = FrozenLakeCustom(
    map_name=args["map_name"], is_slippery=args["is_slippery"],
    render_mode=args["render_mode"])

simulator = FrozenLakeSimulator(env.P)

V_vit, Q_vit = value_iteration(
    simulator, args["gamma"], args["vit_thres"])
# global Q_vit_g = Q_vit
        
for state in range(simulator.num_states):
    logging.warning(f"\n-> state = {state}")
    logging.warning(f"V[state] = {V_vit[state]:0.4f}")
    for action in range(simulator.num_actions):
        logging.warning(f"Q[state][action] = {Q_vit[state][action]:0.4f}")
    logging.warning(f"best_action={np.argmax(Q_vit[state])}")
    
manager = mp.Manager()
ep_reward_list = manager.list()
Q_mcts_list = manager.list()

def run_trial(i_trial, Q_vit, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    env = FrozenLakeCustom(
        map_name=args["map_name"], is_slippery=args["is_slippery"],
        render_mode=args["render_mode"])

    simulator = FrozenLakeSimulator(env.P)

    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)

    ep_reward_list.append(ep_reward)
    Q_mcts_list.append(Q_mcts)
    return ep_reward

In [3]:
args["update_method"] = "haver"
args["rollout_method"] = ""
args["action_multi"] = 4

print(f"num_trials = {args['num_trials']}")
print(f"action_multi = {args['action_multi']}")
# print(f"mcts_num_trajectories = {args['mcts_num_trajectories']}")


hparam_ucb_scale_list = np.arange(10, 100, 10)
hparam_ucb_scale_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_ucb_scale_list = [2**i for i in range(1, 9)]
args["hparam_ucb_scale"] = 64

hparam_haver_std_list = np.arange(10, 100, 10)
hparam_haver_std_list = [1, 2, 4, 8, 16, 32, 64, 128]
# hparam_haver_std_list = [2**i for i in range(1, 9)]


num_trajectories_list = [200, 500, 1000, 1500, 2000, 2500, 3000]
# num_trajectories_list = [200, 500, 800]
# num_trajectories_list = [2]
best_param_list = []
max_reward_mean_list = []
res_text1 = ""
res_text2 = ""
for num_trajectories in num_trajectories_list:
    print(f"\n-> num_trajectories = {num_trajectories}")
    args["mcts_num_trajectories"] = num_trajectories
    
    best_param = None
    max_reward_mean = -np.inf
    start_time = time.time()
    res_text1 += f"{num_trajectories} "
    res_text2 += f"{num_trajectories} "
    start_time = time.time()
    for hparam_haver_std in hparam_haver_std_list:
        
    
        args["hparam_haver_var"] = hparam_haver_std**2
        print(f"hparam_haver_var = {args['hparam_haver_var']}")
        # print(f"hparam_ucb_scale = {args['hparam_ucb_scale']}")
        
        pool = mp.Pool()
        pool.starmap(run_trial, [(i, Q_vit, args) for i in range(args["num_trials"])])

        reward_mean = np.mean(ep_reward_list)
        reward_std = np.std(ep_reward_list, ddof=1) if len(ep_reward_list) > 1 else 0
        reward_error = reward_std/np.sqrt(args["num_trials"])
        if hparam_haver_std <= 8:
            res_text1 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
        else:
            res_text2 += f"& {reward_mean:0.2f} (\u00B1{reward_error:0.2f}) "
        print(f"reward = {reward_mean:0.2f} +/- {reward_error:0.2f}")

        if reward_mean > max_reward_mean:
            max_reward_mean = reward_mean 
            best_param = hparam_haver_std
    
        ep_reward_list[:] = []
        Q_mcts_list[:] = []
    
        end_time = time.time()
        # print(f"it takes {end_time-start_time:0.4f}")
    
    res_text1 += "\\\\ \n \hline \n"
    res_text2 += "\\\\ \n \hline \n"
    
    # print(f"max_reward_mean = {max_reward_mean:0.2f}")
    print(f"it takes {end_time-start_time:0.4f}")
    
    max_reward_mean_list.append(max_reward_mean)
    best_param_list.append(best_param)

num_trials = 20
action_multi = 4

-> num_trajectories = 200
hparam_haver_var = 1
reward = -38.05 +/- 10.60
hparam_haver_var = 4
reward = -33.20 +/- 10.18
hparam_haver_var = 16
reward = -57.55 +/- 11.09
hparam_haver_var = 64
reward = -76.95 +/- 9.64
hparam_haver_var = 256
reward = -71.90 +/- 10.20
hparam_haver_var = 1024
reward = -71.90 +/- 10.20
hparam_haver_var = 4096
reward = -52.55 +/- 11.12
hparam_haver_var = 16384
reward = -67.10 +/- 10.60
it takes 18.5344

-> num_trajectories = 500
hparam_haver_var = 1
reward = -33.15 +/- 10.22
hparam_haver_var = 4
reward = -28.25 +/- 9.64
hparam_haver_var = 16
reward = -13.70 +/- 6.68
hparam_haver_var = 64
reward = -18.55 +/- 7.95
hparam_haver_var = 256
reward = -71.90 +/- 10.20
hparam_haver_var = 1024
reward = -67.05 +/- 10.61
hparam_haver_var = 4096
reward = -67.05 +/- 10.61
hparam_haver_var = 16384
reward = -62.20 +/- 10.90
it takes 46.6414

-> num_trajectories = 1000
hparam_haver_var = 1
reward = -28.25 +/- 9.64
hparam_haver_var = 4
reward =

Process ForkPoolWorker-118:
Process ForkPoolWorker-109:
Process ForkPoolWorker-138:
Process ForkPoolWorker-102:
Process ForkPoolWorker-98:
Process ForkPoolWorker-172:
Process ForkPoolWorker-187:
Process ForkPoolWorker-203:
Process ForkPoolWorker-327:
Process ForkPoolWorker-324:
Process ForkPoolWorker-159:
Process ForkPoolWorker-308:
Process ForkPoolWorker-45:
Process ForkPoolWorker-168:
Process ForkPoolWorker-71:
Process ForkPoolWorker-135:
Process ForkPoolWorker-123:
Process ForkPoolWorker-134:
Process ForkPoolWorker-53:
Process ForkPoolWorker-104:
Process ForkPoolWorker-153:
Process ForkPoolWorker-112:
Process ForkPoolWorker-177:
Process ForkPoolWorker-74:
Process ForkPoolWorker-92:
Process ForkPoolWorker-263:
Process ForkPoolWorker-110:
Process ForkPoolWorker-195:
Process ForkPoolWorker-80:
Process ForkPoolWorker-55:
Process ForkPoolWorker-125:
Process ForkPoolWorker-61:
Traceback (most recent call last):
Process ForkPoolWorker-76:
Process ForkPoolWorker-161:
Process ForkPoolWorker-

KeyboardInterrupt: 

KeyboardInterrupt
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/synchronize.py", line 102, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/synchronize.py", line 102, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/synchronize.py", line 102, in __enter__
    return self._semlock.__enter__()
  File "/tmp/ipykernel_2489687/1569177164.py", line 41, in run_trial
    Q_mcts, ep_reward = run_mcts_trial(env, simulator, Q_vit, i_trial, args)
KeyboardInterrupt
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/synchronize.py", line 102, in __enter__
    return self._semlock.__enter__()
  File "/home/tnn/miniconda3/envs/tnn1/lib/python3.7/site-packages/multiprocess/synchronize.py", line 102, in __enter__
    return self._semlock.__enter__

In [ ]:
print(res_text1)
print(res_text2)